# Transfer AE

Use a pre-trained autoencoder as tranfer-learning preprocessing (feature extractor).

## What?

![transferAE](transferAE.jpg)

## Why?

Je vous le donne en 5 :

* Toute donnée est bonne à prendre (data driven).

* Les datasets marchent main dans la main.

* L'espace des possibles est extrêmement grand. La fenêtre ouverte par les données est très précises au final donc toute données affine un modèle (e.g. autoencoder). Autrement dit, l'ensemble des signaux plausibles est très petit.

* En gros on compresse des données pour les emmener avec soit.

* Ainsi on peut tirer un avantage de données de sources très variées (different shapes and tasks).

Pas forcément besoin d'une plasticité élevée pour bien généraliser

Plasticité : taux d'apprentissage de l'encoder (notamment pendant l'apprentissage d'un modèle)


//Test sur AutoDL ?
//Est ce que avec un 3D AE train sur tous les datasets ca peut marcher ? (Padding)
//Ou il faut être domaine specific ?
//Il pourrait peut-être maîtriser plusieurs domaine grâce à une certaine plasticité. On le re train un peu sur le nouveau //dataset pour l'adapter et lui rappeler le bon domaine ?

## How?

# 1. Input shaper

Input data of different shapes.

* Soit un réseau capable de gerer des entrées de taille différentes;
* Soit un processing qui les mets au même format (padding and/or resizing)

In [44]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [45]:
x_train = np.array(list(map(lambda x: np.expand_dims(x, axis=2), x_train)))
x_test = np.array(list(map(lambda x: np.expand_dims(x, axis=2), x_test)))

# 2. Autoencoder

Unsupervised, no task. Train on every datasets.

Plutôt que d'avoir un modèle génératif qui permet de faire de l'augmentation de données, celui-ci apprend une représentation abstraite des données.

In [46]:
from IPython.display import Image, SVG
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
Model = tf.keras.models.Model
Sequential = tf.keras.models.Sequential
Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape = tf.keras.layers.Input, tf.keras.layers.Dense, tf.keras.layers.Conv2D, tf.keras.layers.MaxPooling2D, tf.keras.layers.UpSampling2D, tf.keras.layers.Flatten, tf.keras.layers.Reshape
regularizers = tf.keras.regularizers

In [48]:
#row_shape, col_shape = x_train.shape[1:]
#channel_shape = 1
#input_shape = (row_shape, col_shape, channel_shape)
input_shape = x_train.shape[1:]

In [49]:
autoencoder = Sequential()

# Encoder Layers
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), strides=(2,2), activation='relu', padding='same'))

# Flatten encoding for visualization
autoencoder.add(Flatten())
autoencoder.add(Reshape((4, 4, 8)))

# Decoder Layers
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 4, 4, 8)           584       
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 4, 4, 8)           0         
__________

In [50]:
#encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('flatten_1').output)
#encoder.summary()

In [51]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=128,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 47s 781us/step - loss: 27.9573 - val_loss: 17.8638
Epoch 2/100
60000/60000 [==============================] - 43s 721us/step - loss: 16.4896 - val_loss: 15.3050
Epoch 3/100
60000/60000 [==============================] - 43s 721us/step - loss: 14.7836 - val_loss: 14.1191
Epoch 4/100
60000/60000 [==============================] - 42s 705us/step - loss: 13.8792 - val_loss: 13.5888
Epoch 5/100
60000/60000 [==============================] - 42s 703us/step - loss: 13.3217 - val_loss: 12.9245
Epoch 6/100
60000/60000 [==============================] - 42s 693us/step - loss: 12.9134 - val_loss: 12.5840
Epoch 7/100
60000/60000 [==============================] - 42s 697us/step - loss: 12.6076 - val_loss: 12.3650
Epoch 8/100
60000/60000 [==============================] - 44s 739us/step - loss: 12.3671 - val_loss: 12.1008
Epoch 9/100
60000/60000 [==============================] - 43s 710us/s

KeyboardInterrupt: 

In [ ]:
input_img = Input(shape=(input_dim,))
encoder_layer1 = autoencoder.layers[0]
encoder_layer2 = autoencoder.layers[1]
encoder_layer3 = autoencoder.layers[2]
encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))

encoder.summary()

# 3. Model

Un discriminant unique pour chaque problème de chaque dataset.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)